In [3]:
#Loading required libraries.
import pandas as pd
import numpy as np
import re
import string
from pprint import pprint
from collections import Counter

#Loading nltk library and its packages for text processing.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

#Downloading some necessary word lists.
#nltk.download('wordnet')
#nltk.download('stopwords')

#Downloading all dependencies and corpora.
#nltk.download()
#nltk.download('all', halt_on_error=False)

#Importing pattern and its dependencies.
import pattern
from pattern.en import tag
from pattern.en import parse
from pattern.en import pprint


In [4]:
#Reading Training and Testing Datasets.
raw_data = pd.read_csv("C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/Data/tweet-sentiment-extraction/train.csv")
#test_data = pd.read_csv("C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/Data/tweet-sentiment-extraction/test.csv")



#Contraction directory.

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}



#Customized stop words.
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 
              'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
              'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'he', 'her', 'here', 
              'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'it', "it's", 'its', 
              'itself', 'just', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'now', 'o', 'of', 'off', 'on', 
              'once', 'only', 'or', 'other', 'our',  'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 
              'shan', "shan't", 'she', "she's", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 
              'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 
              'too', 'under', 'until', 'up', 've', 'very', 'we', 'while', 'with', 'y', 'you', "you'd", "you'll", 
              "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}



#Function for expanding contractions using above dictonary of contractions.
def expand_contractions(sentence, contraction_mapping):
    #Creating a list of contraction keys.
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags = re.IGNORECASE|re.DOTALL)
    
    #Function for expanding the contractions.
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = (contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower()))
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence



#Function for creating a list ofstrings from the input. It creates different elements in the list if there is a space.
#Basically, a sentence will be converted into a list of words.
def words(text): return re.findall(r'\w+', text.lower())

#Reading the file with vocabulary.
WORDS = Counter(words(open('C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/big.txt').read()))

#Function for calculating probability of the given word to find in the vocabulary.
def P(word, N = sum(WORDS.values())): 
    "Probability of `word`."
    #WORDS[word] gives the count of word
    #N is total number of words in the vocabulary.
    return WORDS[word] / N

#Main function which should be called for word correction.
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key = P)

#Function for genetraing all possible real words from given string.
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

#Function for editing given string. This returns words which are one edit away from input string.
#The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit: 
#a simple edit to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter).
#The output of this function can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 
#26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    #Splitting the letters of a word to form a combination.
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

#Function for generating set of words which are two edits away from input string.
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


#Function for converting a list to a string.
def listToString(inlist):
    #Initialize an empty string.
    string = " "
    return (string.join(inlist))



In [5]:

#Function for preprocessing and normalizing text data.
def data_normalization(indata):
    #Setting stop word variable using NLTK package.
    stopWords = set(stopwords.words('english'))
    
    #Creating list of punctuations.
    punctuations = [".", ",", "!", "`", "...", "****", ":", "(", ")", "?", "-", "{", "}", "[", "]", ";", "/", "+", "<", 
               ">", "|", "=" "_", "^", "@", "#", "$", "%", "~", "'", ".."]
    
    #Regular expression tokenizer with only alphanumeric contents.
    Token_Pattern = r'\w+'
    regex_wt = nltk.RegexpTokenizer(pattern = Token_Pattern, gaps = False)

    #Listing out all contractions.
    contaction_keys = list(CONTRACTION_MAP.keys())

    #Declaring list for storing lemmatized tweets.
    lemmatized_words = []
    for i in range(len(indata)):
        #Condition for empty tweets.
        if (str(indata[i]) == "nan"):
            #Fake strink in the blank tweet.
            indata[i] = "NA"
        #Expanding contractions.
        text_segment = indata[i].replace("`", "'")
        expanded_tweets = expand_contractions(text_segment, CONTRACTION_MAP)
        #Removing URL links from tweets.
        expanded_tweets = re.sub(r'http\S+', '', expanded_tweets, flags = re.MULTILINE)
        #print("After contractions: ", expanded_tweets)
        #Tokenizing the words.
        tokens = regex_wt.tokenize(expanded_tweets)
        #Declaring list for lemmatized words.
        lemm_list = []
        for word in tokens:
            #Condition for stopwords.
            #if (word not in stopWords):
            if (word not in stop_words):
                #Condition for punctuations.
                if (word not in punctuations):
                    #Correcting the word.
                    #Not using this function because its not correcting words properly. Need to find more accurate way!
                    #corrected_spell = correction(word)
                    corrected_spell = word
                    #Converting all letters to lower form.
                    corrected_spell = corrected_spell.lower()
                    #Lemmatizing words.
                    lemm_list.append(wordnet_lemmatizer.lemmatize(corrected_spell, pos = "v"))
        #Converting the lemmatized word's (document) list to a string.
        lemm_string = listToString(lemm_list)
        lemmatized_words.append(lemm_string)
    return lemmatized_words


In [6]:
#Function for extracting features from given data. There are three options of extraction methods are given:
#Those are Bag of Words-based frequencies, occurrences, and TF-IDF based features
def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1, 1), min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")
    
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    return (vectorizer, feature_matrix)


#Function for the model accuracy, precision, recall, and F1-score.
def get_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, predicted_labels), 2))
    print('Precision:', np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 2))
    print('Recall:', np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 2))
    print('F1 Score:', np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 2))


#Function for building the confusion matrix for evaluating the model predictions against the actual sentiment 
#labels for the tweets.
def display_confusion_matrix(true_labels, predicted_labels):
    cm = metrics.confusion_matrix(y_true = true_labels, y_pred = predicted_labels)
    cm_frame = pd.DataFrame(cm)
    print(cm_frame)


#Function for getting a detailed classification report per sentiment category (positive, neutral and negative) 
#by displaying the precision, recall, F1-score, and support (number of reviews) for each of the classes.
def display_classification_report(true_labels, predicted_labels):
    report = metrics.classification_report(y_true = true_labels, y_pred = predicted_labels)
    print(report)
    
    


In [7]:
#Raw tweets
raw_tweets = raw_data['text']
#Raw tweets to view.
training_raw_tweets = np.array(raw_tweets[:27000])
testing_raw_tweets = np.array(raw_tweets[27000:])

#Normalizing tweets.
normalized_tweets = data_normalization(raw_data['text'])

#Splitting train and test tweets.
train_tweets = np.array(normalized_tweets[:27000])
test_tweets = np.array(normalized_tweets[27000:])

#Splitting sentiment labels.
labels = raw_data['sentiment']
train_labels = np.array(labels[:27000])
test_labels = np.array(labels[27000:])


In [23]:
#Vectorizing training tweets.
#Building vectorizer model on trainning tweets and extracting features for train tweets.
vectorizer, train_features = build_feature_matrix(train_tweets, feature_type = 'tfidf', ngram_range = (1, 1), 
                                                  min_df = 0.0, max_df = 1.0)

#Extracting features for test tweets
test_features = vectorizer.transform(test_tweets)


In [24]:

print(train_tweets[0])
print(train_features[0])


i would have respond i be go
  (0, 8809)	0.31416420825794106
  (0, 3170)	0.19997868615838085
  (0, 16410)	0.7727597783541493
  (0, 9459)	0.26504438601206154
  (0, 21540)	0.4403443808356867


In [91]:
#Building the SVM classification model.
svm = SGDClassifier(loss = 'hinge')
svm.fit(train_features, train_labels)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [92]:
#Printing the sample tested tweets and predicitons.
for i in range(10):
    print('Tweet:-')
    print(testing_raw_tweets[i])
    print('Actual Labeled Sentiment:', test_labels[i])
    doc_features = test_features[i]
    predicted_sentiment = svm.predict(doc_features)[0]
    print('Predicted Sentiment:', predicted_sentiment, '\n')


Tweet:-
back to work tomoo  day 1 of 5 it will bee.
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 Wow. That`s looks really good. I wish I had some. Was it good?
Actual Labeled Sentiment: positive
Predicted Sentiment: positive 

Tweet:-
  i wasn`t able to go the conference.so I didn`t get the outline.. however I have been doing my own & it`s coming along.
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 I only came across viva la juicy on fri, no testers only body lotion in the store.... but i defo need to get it asap
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 Maybe because you got in 4 hours ago from an awesome night? Nah, that can`t be it.
Actual Labeled Sentiment: neutral
Predicted Sentiment: positive 

Tweet:-
 i miss you  when are you gonna be down in irvine again?? liz and i were just reminiscing about all our adventures.
Actual Labeled Sentiment: negative
Predicted Sentiment: negative 

Tweet:-
 :

In [101]:
#Predicting the sentiments for entire test dataset.
predicted_sentiments = svm.predict(test_features)
get_metrics(true_labels = test_labels, predicted_labels = predicted_sentiments)


Accuracy: 0.74
Precision: 0.74
Recall: 0.74
F1 Score: 0.74


In [94]:
#Binary
#Displaying report.
display_classification_report(true_labels = test_labels, predicted_labels = predicted_sentiments, classes = [1,0])


              precision    recall  f1-score   support

    negative       0.68      0.64      0.66       116
     neutral       0.73      0.77      0.75       211
    positive       0.81      0.78      0.79       154

    accuracy                           0.74       481
   macro avg       0.74      0.73      0.73       481
weighted avg       0.74      0.74      0.74       481



In [159]:
#Frequency
#Displaying report.
display_classification_report(true_labels = test_labels, predicted_labels = predicted_sentiments, classes = [1,0])


              precision    recall  f1-score   support

    negative       0.66      0.60      0.63       116
     neutral       0.71      0.78      0.74       211
    positive       0.81      0.75      0.78       154

    accuracy                           0.73       481
   macro avg       0.73      0.71      0.72       481
weighted avg       0.73      0.73      0.73       481



In [165]:
#Tfid
#Displaying report.
display_classification_report(true_labels = test_labels, predicted_labels = predicted_sentiments, classes = [1,0])


              precision    recall  f1-score   support

    negative       0.68      0.57      0.62       116
     neutral       0.68      0.78      0.73       211
    positive       0.79      0.73      0.76       154

    accuracy                           0.71       481
   macro avg       0.72      0.69      0.70       481
weighted avg       0.72      0.71      0.71       481



In [86]:
#from sklearn.ensemble import RandomForestClassifier

#Building Random Forest classification model.
#rf = RandomForestClassifier(n_estimators = 1000)
#, max_depth = 3
#rf.fit(train_features, train_labels)




mnb = MultinomialNB()
mnb.fit(train_features, train_labels)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
#Printing the sample tested tweets and predicitons.
for i in range(10):
    print('Tweet:-')
    print(testing_raw_tweets[i])
    print('Actual Labeled Sentiment:', test_labels[i])
    doc_features = test_features[i]
    predicted_sentiment = mnb.predict(doc_features)[0]
    print('Predicted Sentiment:', predicted_sentiment, '\n')


Tweet:-
back to work tomoo  day 1 of 5 it will bee.
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 Wow. That`s looks really good. I wish I had some. Was it good?
Actual Labeled Sentiment: positive
Predicted Sentiment: positive 

Tweet:-
  i wasn`t able to go the conference.so I didn`t get the outline.. however I have been doing my own & it`s coming along.
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 I only came across viva la juicy on fri, no testers only body lotion in the store.... but i defo need to get it asap
Actual Labeled Sentiment: neutral
Predicted Sentiment: negative 

Tweet:-
 Maybe because you got in 4 hours ago from an awesome night? Nah, that can`t be it.
Actual Labeled Sentiment: neutral
Predicted Sentiment: neutral 

Tweet:-
 i miss you  when are you gonna be down in irvine again?? liz and i were just reminiscing about all our adventures.
Actual Labeled Sentiment: negative
Predicted Sentiment: neutral 

Tweet:-
 : 

In [88]:
#Predicting the sentiments for entire test dataset.
predicted_sentiments = mnb.predict(test_features)


#Binary Random Forest.
#Displaying report.
display_classification_report(true_labels = test_labels, predicted_labels = predicted_sentiments, classes = [1,0])



              precision    recall  f1-score   support

    negative       0.62      0.55      0.58       116
     neutral       0.63      0.69      0.66       211
    positive       0.71      0.66      0.68       154

    accuracy                           0.65       481
   macro avg       0.65      0.64      0.64       481
weighted avg       0.65      0.65      0.65       481



In [11]:

#Function for predicting tweet.
def sentiment_detector(text, vectorizer_model, model):
    #Normalizing the text.
    normalized_text = data_normalization(text)
    tweet = np.array(normalized_text)
    print(tweet)
    #Extracting features from tweet.
    tweet_features = vectorizer_model.transform(tweet)
    features = tweet_features
    #Predicting using trained model.
    prediction = model.predict(features)[0]
    return prediction


In [95]:

text = ['Packing is no fun at all good thing I have the united on to keep me going... still no fun']
sentiment_detector(text, vectorizer_model = vectorizer, model = svm)


['pack be no fun good thing i have unite keep go still no fun']


'positive'

In [102]:
#Confusion matrix.
display_confusion_matrix(test_labels, predicted_sentiments)


     0    1   2
0  120   23  11
1   24  163  24
2    5   37  74
    0    1    2
0  74   37    5
1  24  163   24
2  11   23  120


In [63]:
#Function for data preprocessing, splitting and normalization.
#Inputs: raw list of tweets, percentage split of training, validation and testing set.
def set_preperation(raw_data, trainset_ratio, valset_ratio, testset_ratio):
    
    raw_tweets = raw_data['text']
    #Total number of observations.
    total_set = len(raw_tweets)
    #Splitting values.
    trainset_len = round(total_set * (trainset_ratio/100))
    valset_len = round(total_set * (valset_ratio/100))
    testset_len = round(total_set * (testset_ratio/100))
    
    #Raw tweets to view.
    training_raw_tweets = np.array(raw_tweets[: trainset_len])
    val_raw_tweets = np.array(raw_tweets[trainset_len : (trainset_len + valset_len)])
    testing_raw_tweets = np.array(raw_tweets[(trainset_len + valset_len) :])

    #Normalizing tweets.
    normalized_tweets = data_normalization(raw_data['text'])

    #Splitting train and test tweets.
    train_tweets = np.array(normalized_tweets[: trainset_len])
    val_tweets = np.array(normalized_tweets[trainset_len : (trainset_len + valset_len)])
    test_tweets = np.array(normalized_tweets[(trainset_len + valset_len) :])

    #Splitting sentiment labels.
    #labels = raw_data['sentiment']
    labels = raw_data['user_verified']
    train_labels = np.array(labels[: trainset_len])
    val_labels = np.array(labels[trainset_len : (trainset_len + valset_len)])
    test_labels = np.array(labels[(trainset_len + valset_len) :])
    
    return train_tweets, val_tweets, test_tweets, train_labels, val_labels, test_labels



#Function for feature extraction from all three sets. Model is built on only training set.
#Input: training, validation and testing set and feature_type (default 'binary').
def feature_extraction(train_tweets, val_tweets, test_tweets, feature_type = 'binary'):
    #Vectorizing training tweets.
    #Building vectorizer model on trainning tweets and extracting features for train tweets.
    vectorizer, train_features = build_feature_matrix(train_tweets, feature_type = feature_type, 
                                                      ngram_range = (1, 1), 
                                                      min_df = 0.0, max_df = 1.0)

    #Extracting features for validation and test tweets
    val_features = vectorizer.transform(val_tweets)
    test_features = vectorizer.transform(test_tweets)
    
    return train_features, val_features, test_features




In [43]:
#Preprocessing and splitting the dataset.
train_tweets, val_tweets, test_tweets, train_labels, val_labels, test_labels = set_preperation(raw_tweets, 
                                                                                               trainset_ratio = 80, 
                                                                                               valset_ratio = 10, 
                                                                                               testset_ratio = 10)

#Extracting features.
train_features, val_features, test_features = feature_extraction(train_tweets, val_tweets, test_tweets, 
                                                                 feature_type = 'binary')



In [31]:
#Building the SVM classification model.
svm = SGDClassifier(loss = 'hinge')
svm.fit(train_features, train_labels)


In [ ]:
import parfit.parfit as pf
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score

grid = {
    'penalty': ['l1', 'l2', 'elasticnet']
}

svm = SGDClassifier(loss = 'hinge')
paramGrid = ParameterGrid(grid)
bestModel, bestScore, allModels, allScores = pf.bestFit(SGDClassifier(), paramGrid,
                                                        train_features, train_labels, val_features, val_labels, 
                                                        metric = roc_auc_score, 
                                                        scoreLabel = "AUC")
#bestScore = 'max',
print(bestModel, bestScore)

In [97]:
get_metrics(test_labels, predicted_sentiments)

Accuracy: 0.74
Precision: 0.74
Recall: 0.74
F1 Score: 0.74
